In [4]:
import warnings
warnings.filterwarnings(action='ignore')
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [10]:
option = Options()
option.add_argument('--window-size=1800,1000')
driver = webdriver.Chrome('./chromedriver.exe', options=option)
driver.implicitly_wait(5) 

# 파일을 쓰기용으로 open한다.
file = open('review.txt', 'w', -1, 'utf-8')

driver.get('https://comic.naver.com/comment/comment?titleId=721433&no=197')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 전체 댓글을 연다.
driver.find_element_by_xpath('//*[@id="cbox_module_wai_u_cbox_sort_option_tab2"]').click()
time.sleep(1)

# 1페이지 댓글을 읽는다.
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
reviews = soup.select('.u_cbox_contents')
for review in reviews:
    file.write(review.text)

count = 2
# 댓글 페이지 그룹 전체 개수가 몇 개인지 모르기 때문에 무한 루프를 돌린다.
while True:
    page = soup.select('div.u_cbox_page_wrap > a.u_cbox_page')
    file.write('댓글 페이지 개수: {}\n'.format(len(page)))
    # 현재 댓글 그룹의 페이지 개수만큼 반복하며 댓글을 읽어들인다.
    for i in range(len(page)):
        # 다음 댓글 페이지를 연다.
        driver.find_element_by_css_selector(
            '#cbox_module > div > div.u_cbox_paginate > div > a:nth-child(' + str(i + 4) + ')'
        ).click()
        time.sleep(1)
        # 다음 댓글 페이지의 댓글을 읽는다.
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        reviews = soup.select('span.u_cbox_contents')
        for review in reviews:
            file.write(review.text)
        file.write('댓글 {}페이지'.format(count))
        count += 1

    # '다음'을 클릭해서 다음 댓글 페이지 그룹을 연다.
    try:
        driver.find_element_by_css_selector(
            '#cbox_module > div > div.u_cbox_paginate > div > a:nth-child(13)'
        ).click()
        time.sleep(1)
    except:
        break

    # 다음 댓글 페이지 그룹의 첫 번째 페이지 리뷰를 읽는다.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    reviews = soup.select('span.u_cbox_contents')
    for review in reviews:
        file.write(review.text)
    file.write('댓글 {}페이지'.format(count))
    count += 1    

file.close()

//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[1]/div[1]/div/div[4]/a